## This notebook contains the code to implement next word prediction model using RNNs(LSTM) 

### First we will mount to the Google Drive to read the data files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras.utils import to_categorical
from keras.models import Model
import matplotlib.pyplot as plt
import re
import string
import numpy as np
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm_notebook

### I have used Harry Potter Novels as the corpus, but you are free to choose any text corpus

#### Due to memory contraints, I trained it only on one part by setting max_files to 1

In [4]:
def read_text_file(file_path):
    with open(file_path, 'r',encoding="UTF-8") as f:
        return f.read()
    
training_text = ""
# iterate through all files
path = "/content/drive/MyDrive/Colab Notebooks/NextWordPrediction/Data/"
max_file = 1
count = 0
for file in os.listdir(path):
    count+=1
    if count>max_file:
        break
    print(file)
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = path+file
        training_text+=read_text_file(file_path)

Book 3 - The Prisoner of Azkaban.txt


In [5]:
len(training_text)

676978

#### Remove unwanted junk characters and tokenize the data

In [6]:
cleaned = re.sub(r'\W+', ' ', training_text).lower()
tokens = word_tokenize(cleaned)

#### Here, you need to specify the window length to divide sequences for training 

In [7]:
train_len = 4
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

#### You can see below how the sequences are formed with a window lenght of 4

In [8]:
text_sequences[0:10]

[['owl', 'post', 'harry', 'potter'],
 ['post', 'harry', 'potter', 'was'],
 ['harry', 'potter', 'was', 'a'],
 ['potter', 'was', 'a', 'highly'],
 ['was', 'a', 'highly', 'unusual'],
 ['a', 'highly', 'unusual', 'boy'],
 ['highly', 'unusual', 'boy', 'in'],
 ['unusual', 'boy', 'in', 'many'],
 ['boy', 'in', 'many', 'ways'],
 ['in', 'many', 'ways', 'for']]

#### Now, you need to tokenize the data to convert the above tokens into unique indexes

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
train_sequences = tokenizer.texts_to_sequences(text_sequences)
print('Found %s unique tokens.' % len(tokenizer.word_counts))
vocab =  len(tokenizer.word_counts)+1

Found 7895 unique tokens.


In [10]:
n_sequences = np.empty([len(train_sequences),train_len], dtype='int32')
for i in range(len(train_sequences)):
    n_sequences[i] = train_sequences[i]

x = n_sequences[:,:-1]
y = n_sequences[:,-1]
#y_cat = to_categorical(y, num_classes=vocab)
#seq_len = train_inputs.shape[1]
#train_inputs.shape

In [11]:
y = to_categorical(y,num_classes=vocab)

#### Now, it is time to build the super cool Next Word Prediction Model

##### Model Architecure is configurable and you are free to try and figure out different structures and hyper parameters

##### Below architecure is just one way to train the model 

In [111]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocab,10, input_length=3))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# compile network
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(x,y_cat,epochs=,verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 10)             78960     
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 50)             12200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 7896)              402696    
Total params: 516,606
Trainable params: 516,606
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(x, y,validation_split=0.05,batch_size=128, epochs=50)

Epoch 1/50
871/871 [==============================] - 26s 19ms/step - loss: 6.7670 - accuracy: 0.0462 - val_loss: 6.4687 - val_accuracy: 0.0459
Epoch 2/50
871/871 [==============================] - 14s 16ms/step - loss: 6.3637 - accuracy: 0.0539 - val_loss: 6.1788 - val_accuracy: 0.0711
Epoch 3/50
871/871 [==============================] - 14s 16ms/step - loss: 6.0225 - accuracy: 0.0896 - val_loss: 5.9245 - val_accuracy: 0.1040
Epoch 4/50
871/871 [==============================] - 14s 16ms/step - loss: 5.7334 - accuracy: 0.1174 - val_loss: 5.7409 - val_accuracy: 0.1196
Epoch 5/50
871/871 [==============================] - 14s 16ms/step - loss: 5.5188 - accuracy: 0.1340 - val_loss: 5.6835 - val_accuracy: 0.1259
Epoch 6/50
871/871 [==============================] - 14s 16ms/step - loss: 5.3674 - accuracy: 0.1423 - val_loss: 5.6638 - val_accuracy: 0.1353
Epoch 7/50
871/871 [==============================] - 15s 17ms/step - loss: 5.2494 - accuracy: 0.1482 - val_loss: 5.6512 - val_accuracy:

#### Okay, the wait is over, lets predict some words

##### Predict is used to predict the next word

##### Generate text is used to generate some random text by using the starting few words. 

In [75]:
from keras.preprocessing.sequence import pad_sequences

def predict(model,text):
  text = text.lower()
  test_sequences = tokenizer.texts_to_sequences([text])
  pad_encoded = pad_sequences(test_sequences, maxlen=3, truncating='pre')
  predicted_word = model.predict(pad_encoded)
  label = predicted_word.argmax(axis=1)
  return tokenizer.index_word[label[0]]

def generate_text(model, start_text, max_words =25):
  output = start_text
  for i in range(max_words):
      output+=" "+predict(model,output)
  return output
  

In [108]:
predict(model,"Harry took the invisibility")

'cloak'

In [109]:
generate_text(model, "Harry took the invisibility", max_words =25)

'Harry took the invisibility cloak and the decision of the committee member from binder a tadpole and the next day and began to chop the room of the crowd'

In [110]:
generate_text(model, "Harry took the invisibility", max_words =50)

'Harry took the invisibility cloak and the decision of the committee member from binder a tadpole and the next day and began to chop the room of the crowd and lanterns hopping around at the moment he was recognizable at the back of the room unsmiling and watchful the castle and then propelled itself'